In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec
import cantera as ct
from sklearn.linear_model import LinearRegression
import os
import concurrent.futures

def reverse_Arrhenius(a):   
    reaction_idx=a
    gas = ct.Solution(cti_file, 'gas')
    surf = ct.Interface(cti_file,'surface1', [gas])

    t=np.linspace(100,800, 50)
    k_forward=np.zeros(len(t))
    k_reverse=np.zeros(len(t))
    for i in range(len(t)):
      gas.TPX =t[i]+273, ct.one_atm, 'Ar:73.9695'
      surf.TP = t[i]+273, ct.one_atm
      surf.coverages = {'site(100)':1}
      k_reverse[i]=surf.reverse_rate_constants[reaction_idx]
      k_forward[i]=surf.forward_rate_constants[reaction_idx]
   
    T_reciprok=1/(t+273)*1000    
    log_k_forward=np.log(k_forward)
    log_k_reverse=np.log(k_reverse)
    log_k_reverse=log_k_reverse.reshape(len(k_reverse),1)
    T_reciprok=T_reciprok.reshape(len(T_reciprok),1)

    #Linear regression of an Arrhenius expression to the calculated rate constant
    model =LinearRegression(fit_intercept=True)
    model.fit(T_reciprok,log_k_reverse)
    yfit=model.predict(log_k_reverse)
    Ea_R=-float(model.coef_[0])
    ln_k0=float(model.intercept_[0])
    activation_barrier=np.round(Ea_R*8.314,3)
    pre_exponential=float(np.format_float_scientific(np.exp(ln_k0)*10,precision=3))
    #print(surf.reaction_equations()[a])
    #print(pre_exponential,activation_barrier)
    return pre_exponential, activation_barrier

cti_file='just_to_flip.yaml'
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 373, ct.one_atm, 'Ar:73.9695'

# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = 273, ct.one_atm
surf.coverages = {'site(100)':1}

rev_params=reverse_Arrhenius(0)
print(rev_params[0]/1e12)
print(rev_params[1])

0.7577
25.748


In [19]:
cti_file='Multifacet_mechanism_flipped_reactions_covdep.yaml'
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 373, ct.one_atm, 'Ar:73.9695'

# import the surface model
surf = ct.Interface(cti_file,'surface4', [gas])
surf.TP = 273, ct.one_atm
surf.coverages = {'site(110)':1}

for i in range(surf.n_reactions):
    print(surf.delta_standard_enthalpy[i]/1e6)

-477.54053895945714
-144.37562267087173
-164.15446041469198
51.16779084690005
88.1142076859575
